### 1.Python script to fetch hourly AQI, temperature, humidity, and pollution levels.

In [4]:
import pandas as pd
import requests
import time
import datetime

In [3]:
##CITY = "Lagos"
##LAT = 6.5244,
##LON = 3.3792
##API_KEY = 915cbda089714672f9fa8cc2ef1cb053

In [2]:
url = "http://api.openweathermap.org/data/2.5/weather?q=Lagos&appid=915cbda089714672f9fa8cc2ef1cb053"
url = "http://api.openweathermap.org/data/2.5/air_pollution?lat={"6.5244"}&lon={"3.3792"}&appid={"915cbda089714672f9fa8cc2ef1cb053"}"

In [37]:
def fetch_weather():  
    """Fetch current weather data (temperature & humidity)"""
    url = f"http://api.openweathermap.org/data/2.5/weather?q={"Lagos"}&appid={"915cbda089714672f9fa8cc2ef1cb053"}&units=metric"
    res = requests.get(url).json()

    return{
        "city": res["name"],
        "temp_c": res["main"]["temp"],
        "humidity": res["main"]["humidity"],
        "timestamp": datetime.datetime.now().isoformat()

    }

weather = fetch_weather()
print(weather)


{'city': 'Lagos', 'temp_c': 23.03, 'humidity': 91, 'timestamp': '2025-08-19T23:03:25.784875'}


In [ ]:
### AQI stands for Air Quality Index = 
### An AQI of 50 or below (green) is generally considered good, while an AQI above 100 indicates unhealthy conditions

In [39]:
def fetch_aqi():
    """Fetch current air quality index + pollutants"""
    url = f"http://api.openweathermap.org/data/2.5/air_pollution?lat={"6.5244"}&lon={"3.3792"}&appid={"915cbda089714672f9fa8cc2ef1cb053"}"
    res = requests.get(url).json()
    data = res["list"][0]
    
    return {
        "aqi": data["main"]["aqi"],   # 1=Good, 2=Fair, 3=Moderate, 4=Poor, 5=Very Poor
        "pm2_5": data["components"]["pm2_5"],
        "pm10": data["components"]["pm10"],
        "o3": data["components"]["o3"],
        "co": data["components"]["co"],
        "timestamp": datetime.datetime.now().isoformat()
    }

aqi = fetch_aqi()
print(fetch_aqi())

{'aqi': 1, 'pm2_5': 1.78, 'pm10': 6.46, 'o3': 34.13, 'co': 155.55, 'timestamp': '2025-08-19T23:16:44.602248'}


In [40]:
### fetch Hourly in Intervals (3600 seconds = 1 hour)

In [ ]:
def collect_hourly_data(interval=3):
    """Collect weather + AQI every few seconds (default=3)"""
    while True:
        weather = fetch_weather()
        aqi = fetch_aqi()
        
        record = {
            "city": weather["city"],
            "temp_c": weather["temp_c"],
            "humidity": weather["humidity"],
            "aqi": aqi["aqi"],
            "pm2_5": aqi["pm2_5"],
            "pm10": aqi["pm10"],
            "o3": aqi["o3"],
            "co": aqi["co"],
            "timestamp": weather["timestamp"]
        }
        print(record)
        time.sleep(interval)

collect_hourly_data(interval=3)

### FINAL RESULT (SINGLE LOOP)

In [7]:
def fetch_weather():  
    """Fetch current weather data (temperature & humidity)"""
    url = f"http://api.openweathermap.org/data/2.5/weather?q={"Lagos"}&appid={"f2d87e47b5cc7656fdde646e4ea16224"}&units=metric"
    res = requests.get(url).json()

    return{
        "city": res["name"],
        "temp_c": res["main"]["temp"],
        "humidity": res["main"]["humidity"],
        "timestamp": datetime.datetime.now().isoformat()

    }

def fetch_aqi():
    """Fetch current air quality index + pollutants"""
    url = f"http://api.openweathermap.org/data/2.5/air_pollution?lat={"6.5244"}&lon={"3.3792"}&appid={"f2d87e47b5cc7656fdde646e4ea16224"}"
    res = requests.get(url).json()
    data = res["list"][0]
    
    return {
        "aqi": data["main"]["aqi"],   # 1=Good, 2=Fair, 3=Moderate, 4=Poor, 5=Very Poor
        "pm2_5": data["components"]["pm2_5"],
        "pm10": data["components"]["pm10"],
        "o3": data["components"]["o3"],
        "co": data["components"]["co"],
        "timestamp": datetime.datetime.now().isoformat()
    }

weather = fetch_weather()
aqi = fetch_aqi()
print(weather)
print(fetch_aqi())

{'city': 'Lagos', 'temp_c': 27.62, 'humidity': 69, 'timestamp': '2025-08-22T11:28:00.664267'}
{'aqi': 1, 'pm2_5': 3.58, 'pm10': 13.06, 'o3': 47.21, 'co': 256.13, 'timestamp': '2025-08-22T11:28:02.556440'}


### 2.AQI Categories and Detecting Spikes

In [1]:
###AQI scale is simple:

#AQI Value	Category
#1	         Good
#2	         Fair
#3	        Moderate
#4	        Poor
#5	        Very Poor

In [9]:
def categorize_aqi(aqi_value):
    categories = {
        1: "Good",
        2: "Fair",
        3: "Moderate",
        4: "Poor",
        5: "Very Poor"
    }
    return categories.get(aqi_value, "Unknown")


def detect_spikes(current, previous, threshold=20):
    """
    Detect spikes in pollutants.
    threshold = % increase (default 20%)
    """
    spikes = {}
    for key in ["pm2_5", "pm10", "o3", "co"]:
        if previous and previous.get(key, 0) > 0: 
            change = ((current[key] - previous[key]) / previous[key]) * 100
            if change >= threshold:
                spikes[key] = round(change, 2)
    return spikes

In [10]:
previous_record = {
    "pm2_5": 1.78, "pm10": 6.46, "o3": 34.13, "co": 155.55
}
current_record = {
    "pm2_5": 3.58, "pm10": 13.06, "o3": 47.21, "co": 256.13
}

# Categorize AQI

aqi_value = 1
category = categorize_aqi(aqi_value)
print("AQI Category:", category)

# Detect spikes

spikes = detect_spikes(current_record, previous_record, threshold=30)
print("Spikes detected:", spikes)

AQI Category: Good
Spikes detected: {'pm2_5': 101.12, 'pm10': 102.17, 'o3': 38.32, 'co': 64.66}


### LOAD INTO postgreSQL

In [11]:
import psycopg2
from psycopg2.extras import RealDictCursor

In [17]:
def get_connection():
    return psycopg2.connect(
        host="localhost",
        database="First_ETL",   
        user="postgres",           
        password="",
        port=5432,
    )

In [18]:
def insert_reading(record):
    conn = get_connection()
    cur = conn.cursor()
    query = """
    INSERT INTO sensor_readings 
    (city, temperature_c, humidity, aqi, aqi_category, pm2_5, pm10, o3, co, recorded_at)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    RETURNING id;
    """
    cur.execute(query, (
        record["city"], record["temp_c"], record["humidity"], 
        record["aqi"], record["aqi_category"],
        record["pm2_5"], record["pm10"], record["o3"], record["co"], 
        record["timestamp"]
    ))
    reading_id = cur.fetchone()[0]
    conn.commit()
    cur.close()
    conn.close()
    return reading_id


def insert_alert(reading_id, alert):
    conn = get_connection()
    cur = conn.cursor()
    query = """
    INSERT INTO alerts 
    (reading_id, alert_type, parameter, old_value, new_value, change_percent, message)
    VALUES (%s, %s, %s, %s, %s, %s, %s);
    """
    cur.execute(query, (
        reading_id, 
        alert["alert_type"], 
        alert["parameter"], 
        alert["old_value"], 
        alert["new_value"], 
        alert["change_percent"], 
        alert["message"]
    ))
    conn.commit()
    cur.close()
    conn.close()

In [23]:
import psycopg2

def get_connection():
    return psycopg2.connect(
        host="localhost",
        database="First_ETL", 
        user="postgres",            
        password="Pastore9717",
        port=5432
    );
try:
    conn = get_connection()
    cur = conn.cursor()
    cur.execute("SELECT NOW();")
    print("✅ DB Connected. Time:", cur.fetchone())
    cur.close()
    conn.close()
except Exception as e:
    print("❌ DB Error:", e)

✅ DB Connected. Time: (datetime.datetime(2025, 8, 22, 13, 32, 47, 336959, tzinfo=datetime.timezone(datetime.timedelta(seconds=3600))),)
